In [ ]:
# Import cortex python library
from cortex4py.api import Api
from cortex4py.query import *
import json
import time

In [ ]:
# Your API key for Cortex. Create it in Cortex
api_key = ''

In [ ]:
# Create the cortex client
api = Api('http://localhost:9001', api_key, verify_cert=False)


In [ ]:
# Get the list of enabled analyzers
for analyzer in api.organizations.get_analyzers():
  print(analyzer.name)


In [ ]:
# Get analyzer that support the specific data type, domain, url, hash etc
for analyzer in api.analyzers.get_by_type('domain'):
    print(analyzer.name)

In [ ]:
# Submit IOC to an analyzer, force=1 is to bypass cache result
# Domain from https://www.malware-traffic-analysis.net/2020/04/15/index.html
vt_job = api.analyzers.run_by_name('VirusTotal_GetReport_3_0', {
    'data': 'raihanchow.us',
    'dataType': 'domain',
    'tlp': 1,
    'pap': 2,
    'message': 'A suspicious domain',

}, force=1)
vt_submission_result = vt_job.json()

In [ ]:
vt_submission_result

In [ ]:
_job = api.jobs.get_by_id(vt_job.id)
_job.status

In [ ]:
_result = api.jobs.get_report_async(vt_job.id).report
_result

In [ ]:
_result['summary']

In [ ]:
# Get all enabled analyzer for an indicator type
def get_analyzer(indicator_type):
    analyzer_set = set()
    for analyzer in api.analyzers.get_by_type(indicator_type):
        analyzer_set.add(analyzer.name)
    return analyzer_set

In [ ]:
# Submitting to multiple analyzers
def analyze(indicator_type, indicator):
    '''
    Parameters
    ----------
    indicator_type: str
        The type of indicator, could be hash, url, domain, ip, file
    indicator: str
        The artifact that is being investigated 
    '''
    
    job_result_dict = {}
    analyzer_set = get_analyzer(indicator_type)
    for analyzer in analyzer_set:
        job = api.analyzers.run_by_name(analyzer, {
            'data': indicator,
            'dataType': indicator_type,
            'tlp': 1,
            'pap': 1, 
            'message': f'Submission for {indicator} on {analyzer}',
        }, force=1)
        job_result_dict.update({analyzer:job.id})
    return job_result_dict

In [ ]:
def get_result(job_result_dict):
    for analyzer, job_id in job_result_dict.items():
        time.sleep(3)
        _result = api.jobs.get_report_async(job_id).report
        print(f"{analyzer}: {_result['summary']}")
        

In [ ]:
job_result_dict = analyze('domain', 'raihanchow.us')

In [ ]:
get_result(job_result_dict)

In [ ]:
get_result(analyze('domain', 'raihanchow.us'))